In [26]:
# !sudo apt-get install tesseract-ocr tesseract-ocr-fas poppler-utils

# !pip install pytesseract pillow pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (5.3.4-1build5).
tesseract-ocr-fas is already the newest version (1:4.1.0-2).
poppler-utils is already the newest version (24.02.0-1ubuntu9.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [27]:
import io
import fitz
import base64
import pytesseract
from PIL import Image
from pathlib import Path

def pdf_page_to_base64(pdf_path: str, page_number: int):
    pdf_document = fitz.open(pdf_path)
    page = pdf_document.load_page(page_number - 1)  # input is one-indexed
    pix = page.get_pixmap()
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    buffer = io.BytesIO()
    img.save(buffer, format="PNG")

    return base64.b64encode(buffer.getvalue()).decode("utf-8")

In [22]:
file_path = "/home/jovyan/projects/data/dining.pdf"
base64_image = pdf_page_to_base64(file_path, 2)
image_data = base64.b64decode(base64_image)

output_path = Path.home() / "projects/data/dining.png"

with open(output_path, "wb") as f:
    f.write(image_data)

print(f"Image saved at {output_path}")

Image saved at /home/jovyan/projects/data/dining.png


In [20]:
# Load the image
image = Image.open(output_path)

# Run OCR with Persian language
# (You may need to install Persian trained data: `sudo apt-get install tesseract-ocr-fas`)
text = pytesseract.image_to_string(image, lang="fas")

print("Extracted Text (Persian):")
print(text)

Extracted Text (Persian):
۳۵

در این داکیومنت یاد میگیریم که:
۰ چگونه بای پرسنل غذا رزوکنیم.
صفحه ی ررو به ما چهاطلاعاتی می دهد.

در این صفحه می توانیم برای پرسئل و با کرت‌های متفرقه غذارزروکنیم.
برای دسترسی به این صفحه از طریق منو سامانه‌ی تغذیه»* رزروغذا ۰" رزرو غذااقدام می‌کنيم
بسیاری از سازمنهایی که می‌خواهند به پرسنل خود حق انتخاب بیش از یک مدل غذا در
وعده‌های غذایی بدهند از این قابلیت استفاده میکنند. همچنین این قابلیت باعث میشه غذا
فقط به تعداد افرادی که قطعی می‌خواهند سرو کنند طبخ شود و این کار باعث صرفه‌جویی در
مصرف مواد غذایی نیز میشود.

در این صفحه لیست کل غذاهایی که در بنمه‌های غذایی تعریف شده به شما تمایش داده
می‌شود و می‌توانید آن ها را تحت شرایطی رزروکنید.

نحوه رزرو غذا در این صفحه:

۰ اگر بخواهیم رزروبرای تاریخ خاصی انجام دهیم از قسمت تاریخ می توانیم انتخاب کنیم و
توسط فلش‌ها به هفته‌های قبل و بعد برویم. دقت کنید که چون در صفحه‌ی رزرو غذا
غذاها را به صورت هفتگی نمایش میدهد هر تاریخی را انتخاب کنید لیست کل غذاهای آن
هفته به شما نمایش داده میشود.

۲ در قسمت مشخصات به صورت

In [29]:
import os
import json
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

# Paths
pdf_path = "/home/jovyan/projects/data/dining.pdf"
output_dir = "/home/jovyan/projects/data/extracted_images"
json_path = "/home/jovyan/projects/data/dining.json"

os.makedirs(output_dir, exist_ok=True)

# Open PDF
doc = fitz.open(pdf_path)

pages_data = []

for page_num in range(len(doc)):
    print("page:", page_num)
    page = doc[page_num]
    text = ""

    # Convert page to image (for OCR)
    pix = page.get_pixmap(dpi=300)
    img_path = os.path.join(output_dir, f"page_{page_num+1}.png")
    pix.save(img_path)

    # OCR Persian text
    text = pytesseract.image_to_string(Image.open(img_path), lang="fas")

    # Extract embedded images
    image_paths = []
    for img_index, img in enumerate(page.get_images(full=True)):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        ext = base_image["ext"]
        image_filename = f"page_{page_num+1}_img_{img_index+1}.{ext}"
        image_output_path = os.path.join(output_dir, image_filename)
        with open(image_output_path, "wb") as f:
            f.write(image_bytes)
        image_paths.append(image_output_path)

    # Store page data
    pages_data.append({
        "page": page_num + 1,
        "text": text.strip(),
        "images": image_paths
    })

# Save JSON
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(pages_data, f, ensure_ascii=False, indent=2)

print(f"✅ Extraction complete! JSON saved to {json_path}")

page: 0
page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
✅ Extraction complete! JSON saved to /home/jovyan/projects/data/dining.json


In [9]:
import os
import json
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

def extract_text_and_images(pdf_path, output_folder, min_width=100, min_height=100, min_filesize=5000):
    """
    Extract Persian OCR text and unique images from each page of a PDF.
    Skips small icons/logos and duplicate images (by size + dimensions).
    Saves extracted images in output_folder.
    Returns a list of dictionaries (one per page).
    """
    doc = fitz.open(pdf_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    seen = set()  # store (width, height, filesize) triples
    results = []  # list of page dictionaries
    
    for page_num in range(len(doc)):
        print("page:", page_num)
        if page_num > 2:
            break
            
        page = doc.load_page(page_num)
        page_dict = {"page": page_num + 1, "text": "", "images": []}
        
        # --- OCR Text (Persian) ---
        # Convert the page to an image for OCR
        pix = page.get_pixmap(dpi=300)
        temp_img_path = os.path.join(output_folder, f"page_{page_num+1}_ocr.png")
        pix.save(temp_img_path)
        page_text = pytesseract.image_to_string(Image.open(temp_img_path), lang="fas")
        page_dict["text"] = page_text.strip()
        
        # --- Extract Images ---
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            width, height = img[2], img[3]
            
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            filesize = len(image_bytes)
            
            # Skip small images
            if width < min_width or height < min_height or filesize < min_filesize:
                continue
            
            # Skip duplicates
            key = (width, height, filesize)
            if key in seen:
                continue
            seen.add(key)
            
            # Save unique image
            image_output_path = os.path.join(
                output_folder, f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            )
            with open(image_output_path, "wb") as f:
                f.write(image_bytes)
            
            page_dict["images"].append({
                "path": image_output_path,
            })
        
        results.append(page_dict)
    
    doc.close()
    return results


# Example usage:
pdf_path = "/home/jovyan/projects/data/helpkasra.pdf"
output_folder = "/home/jovyan/projects/data/extracted_images"

pages_data = extract_text_and_images(pdf_path, output_folder)

# json_path = "/home/jovyan/projects/data/dining.json"
# with open(json_path, "w", encoding="utf-8") as f:
#     json.dump(pages_data, f, ensure_ascii=False, indent=2)

# print(f"✅ JSON saved at {json_path}")

page: 0
page: 1
page: 2
page: 3


In [10]:
pages_data

[{'page': 1,
  'text': '۹5\n\nشرکت کیمیاگران سرزمین رایانه\n\n2\n\nراهنمای عمومی سامانه حضور و غیاب کسرا\n(تردد)',
  'images': [{'path': '/home/jovyan/projects/data/extracted_images/page_1_img_1.jpeg'},
   {'path': '/home/jovyan/projects/data/extracted_images/page_1_img_2.jpeg'}]},
 {'page': 2,
  'text': '(۸\n\nشرکت کیمیاگران سرزمین رایانه\n\nورود به نرم افزار صص_\n\nکسرای من ۳[\n\nجست و جوی صفحات ۳[\n\nمیز کار ۳[\nکار کرد ماهانه |\nکارکرد روزانه 9[\nثبت مجوز از طریق لینک 7[\nدرخواست مجوز ۳[\nدرخواست تردد |\nنمايش مجوز ها 1\nنمایش کاردکس |\n\nتغییر کلمه عبور 1۱\n\nکارتابل 1\n\n1\n\nضرین غایبین ۳[\nآمار روزانه پرسنل 7 صصسصپصپصسصپصپصسصپصپصسصپصرصپصپصپصرصرصر 0',
  'images': []},
 {'page': 3,
  'text': '۹\n\nشرکت کیمیاگران سرزمین رایانه\n\nورود به نرم افزار\nبا وارد نمودن نام کاریری و کلمه عبور میتوانید به نرم افزار وارد شوید . این اطلاعات از طریق مدیر سیستم در اختیار\n\nشما قرار خواهد گرفت در صورتی فراموشی رمز عبور بایستی با مدیر سیستم تماس حاصل نمایید .\n\n6 ورود کاربران\n\n2 8 8۲ ۱۳ ۷ ۰۰

In [12]:
print(pages_data[2]["text"])

۹

شرکت کیمیاگران سرزمین رایانه

ورود به نرم افزار
با وارد نمودن نام کاریری و کلمه عبور میتوانید به نرم افزار وارد شوید . این اطلاعات از طریق مدیر سیستم در اختیار

شما قرار خواهد گرفت در صورتی فراموشی رمز عبور بایستی با مدیر سیستم تماس حاصل نمایید .

6 ورود کاربران

2 8 8۲ ۱۳ ۷ ۰۰ 90 786۵۲۲۵۱۲۱9۵2۴۱۵ موم / نامع 0/۸ 192.1680:106/1290:1۷۵9/1۵۷۱۵ 22 بر 90 ۵ ۵ ی«

جهت کارکرد صحیح صفحات در نرم افزار ؛ از از مرورگرهای 086]و 6۳0۲۵۳6 (۵۵ به بالا )یا ۳۱۳۲6۲0 استفاده نمایید.

آشنایی با سامانه :

با هر مرتبه ورود به نرم افزار راهنمای سیستمی به شما نمایش داده میشود که با کلیک بر روی صفحه میتوانید در آن
لحظه از آن خارج شوید .

جهت عدم نمایش در هنگام ورود مجدد به نرم افزار کافی است یکبار راهنما تا انتها مشاهده گردد و دکمه عدم نمایش
مجدد انتخاب شود .

همچنین با انتخاب دعر [فگا, گزینه آشنایی با سامانه. امکان نمايش مجدد راهنما وجود دارد .
